# Creator

`Creator` represents the logic unit executing on driver to generate a dataframe.

**Input can not have any DataFrame or dataframe-like annotations**, other type of parameters are fine to add

**Acceptable output DataFrame types**: `DataFrame`, `LocalDataFrame`, `pd.DataFrame`, `List[List[Any]]`, `Iterable[List[Any]]`, `EmptyAwareIterable[List[Any]]`, `List[Dict[str, Any]]`, `Iterable[Dict[str, Any]]`, `EmptyAwareIterable[Dict[str, Any]]`

**On the first argument** you can have a parameter with `ExecutionEngine` annotation so Fugue will pass in the current `ExecutionEngine`

Notice

* `ArrayDataFrame` and other local dataframes can't be used as annotation, you must use `LocalDataFrame` or `DataFrme`
* If output type is NOT one of `DataFrame`, `LocalDataFrame` or `pd.DataFrame`, the output schema is unknown, so you must specify that.


## Native Approach

The simplest way, with no dependency on Fugue. You just need to add acceptable annotations to the output.

In [ ]:
from typing import Iterable, Dict, Any, List
import pandas as pd

def create1(n=1) -> pd.DataFrame:
    return pd.DataFrame([[n]],columns=["a"])

# the output has no schema info, so you must specify schema in fugue code
def create2(n=1) -> List[List[Any]]:
    return [[n]]

In [ ]:
from fugue import FugueWorkflow

with FugueWorkflow() as dag:
    dag.create(create1, params={"n":2}).show()
    dag.create(create2, schema="a:int", params={"n":2}).show()

It's very important to see another use case: with `ExecutionEngine`. **This is how you write native Spark code inside Fugue.**

In [ ]:
from fugue import ExecutionEngine, DataFrame
from fugue_spark import SparkExecutionEngine, SparkDataFrame
from typing import Iterable, Dict, Any, List
import pandas as pd

# pay attention to the input and output annotations, they are both general DataFrame
def create(e:ExecutionEngine, n=1) -> DataFrame:
    assert isinstance(e,SparkExecutionEngine) # this extension only works with SparkExecutionEngine
    sdf= e.spark_session.createDataFrame([[n]], schema="a:int")  # this is how you get spark session
    return SparkDataFrame(sdf) # you must wrap as Fugue SparkDataFrame to return

with FugueWorkflow(SparkExecutionEngine) as dag:
    dag.create(create, params={"n":2}).show()

## With Schema Hint

Notice if you are using `DataFrame`, `LocalDataFrame` or `pd.DataFrame` as the output type, you must not have type hint. And the best practice is to use `DataFrame` as the output type.

In [ ]:
from typing import Iterable, Dict, Any, List
import pandas as pd

# schema: a:int
def create(n=1) -> List[List[Any]]:
    return [[n]]

from fugue import FugueWorkflow

with FugueWorkflow() as dag:
    dag.create(create).show()

## Decorator Approach

There is no obvious advantage to use decorator for `Creator`.

In [ ]:
from fugue import FugueWorkflow, creator
from typing import Iterable, Dict, Any, List
import pandas as pd

@creator("a:int")
def create(n=1) -> List[List[Any]]:
    return [[n]]

with FugueWorkflow() as dag:
    dag.create(create).show()

## Interface Approach

All the previous methods are just wrappers of the interface approach. They cover most of the use cases and simplify the usage. But if you want to get all execution context such as partition information, use interface approach.

In the interface approach, type annotations are not necessary, but again, it's good practice to have them.

In [ ]:
from fugue import FugueWorkflow, Creator, DataFrame
from time import sleep
import pandas as pd
import numpy as np


class Array(Creator):
    def create(self) -> DataFrame:
        engine = self.execution_engine
        n = self.params.get_or_throw("n",int)
        return engine.to_df([[n]],"a:int")


with FugueWorkflow() as dag:
    dag.create(Array, params=dict(n=1)).show()

## Some Real Cases to Consider

In the following cases, Fugue does not have built in extensions, but it will be very easy to write them by yourselves

* **Read special data source**, for example you want to load data from a certain service with special API
* **Create mock data in unit test**, you can customize your own mock data generation. And if also with the equality assertion as as `Outputter`, you are able to test your Fugue workflow within one dag.